In [1]:
from dotenv import load_dotenv
from typing import TypedDict, Literal, Annotated
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from IPython.display import display, Image
from langgraph.graph.message import add_messages


load_dotenv()

True

In [2]:
# Init LLM
llm = init_chat_model(model="llama3.1:8b", model_provider="ollama")

# Define state
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Node logic
def chatbot(state: State) -> State:
    return {"messages": state["messages"] + [llm.invoke(state["messages"])]}

# Build graph
builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)
builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)
graph = builder.compile()

In [3]:
# Chat loop
state = None
try:
    while True:
        user_input = input("You: ")
        if user_input.lower() in {"exit", "quit"}:
            print("Exiting chat.")
            break

        # Initialize or update state
        if state is None:
            state = {"messages": [{"role": "user", "content": user_input}]}
        else:
            state["messages"].append({"role": "user", "content": user_input})

        # Get updated state
        state = graph.invoke(state)
        bot_response = state["messages"][-1]

        # Print bot response
        print("Bot:", bot_response["content"])

except KeyboardInterrupt:
    print("\nChat ended by user.")

TypeError: 'AIMessage' object is not subscriptable